In [1]:
import pyodbc
import pandas as pd
import time, sys

In [2]:
driver = '{ODBC Driver 17 for SQL Server}'
# driver = '{SQL Server}'

# localhost
# server = '0.0.0.0,1433'
# Use Docker Toolbox $docker-machine ip
server = '192.168.99.100,1433'
database = 'StackOverflow2017'
username = 'sa'
password = '1StrongPassword'
con = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+password)
cursor = con.cursor()

In [3]:
cursor.execute("SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE='BASE TABLE'")
rows = cursor.fetchall()

for row in rows:
    print(row)

('StackOverflow2017', 'dbo', 'PostTypes', 'BASE TABLE')
('StackOverflow2017', 'dbo', 'PostHistoryTypes', 'BASE TABLE')
('StackOverflow2017', 'dbo', 'Votes', 'BASE TABLE')
('StackOverflow2017', 'dbo', 'Users', 'BASE TABLE')
('StackOverflow2017', 'dbo', 'Posts', 'BASE TABLE')
('StackOverflow2017', 'dbo', 'Comments', 'BASE TABLE')
('StackOverflow2017', 'dbo', 'Badges', 'BASE TABLE')
('StackOverflow2017', 'dbo', 'PostLinks', 'BASE TABLE')
('StackOverflow2017', 'dbo', 'Tags', 'BASE TABLE')
('StackOverflow2017', 'dbo', 'PostHistory', 'BASE TABLE')
('StackOverflow2017', 'dbo', 'LinkTypes', 'BASE TABLE')
('StackOverflow2017', 'dbo', 'VoteTypes', 'BASE TABLE')


In [4]:
cursor.execute("select top 1 * from StackOverflow2017.dbo.Posts")
rows = cursor.fetchall()

for row in rows:
    print(row)

(4, 7, 13, "<p>I want to use a track-bar to change a form's opacity.</p>\n\n<p>This is my code:</p>\n\n<pre><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n\n<p>When I build the application, it gives the following error:</p>\n\n<blockquote>\n  <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code>.</p>\n</blockquote>\n\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\n", None, 1, datetime.datetime(2012, 10, 31, 16, 42, 47, 213000), datetime.datetime(2008, 7, 31, 21, 42, 52, 667000), 37, datetime.datetime(2017, 9, 27, 5, 52, 59, 927000), datetime.datetime(2017, 9, 27, 5, 52, 59, 927000), 'Rich B', 3151675, 8, 0, 1, 528, '<c#><winforms><type-conversion><decimal><opacity>', 'While applying opacity to a form, should we use a decimal or a double value?', 33648)


In [5]:
sql_top5 = 'select top 5 * from StackOverflow2017.dbo.Posts'
data10 = pd.read_sql(sql_top5, con)
data10

,Id,AcceptedAnswerId,AnswerCount,Body,ClosedDate,CommentCount,CommunityOwnedDate,CreationDate,FavoriteCount,LastActivityDate,LastEditDate,LastEditorDisplayName,LastEditorUserId,OwnerUserId,ParentId,PostTypeId,Score,Tags,Title,ViewCount
0,4,7,13,<p>I want to use a track-bar to change a form'...,None,1,2012-10-31 16:42:47.213,2008-07-31 21:42:52.667,37,2017-09-27 05:52:59.927,2017-09-27 05:52:59.927,Rich B,3151675,8,0,1,528,<c#><winforms><type-conversion><decimal><opacity>,"While applying opacity to a form, should we us...",33648
1,6,31,5,<p>I have an absolutely positioned <code>div</...,None,0,NaT,2008-07-31 22:08:08.620,9,2016-03-19 06:10:52.170,2016-03-19 06:05:48.487,Rich B,63550,9,0,1,231,<html><css><css3><internet-explorer-7>,Percentage width child element in absolutely p...,15363
2,7,0,0,<p>An explicit cast to double isn't necessary....,None,0,NaT,2008-07-31 22:17:57.883,0,2012-10-14 11:50:16.703,2012-10-14 11:50:16.703,None,967315,9,4,2,379,None,None,0
3,9,1404,61,<p>Given a <code>DateTime</code> representing ...,None,10,2011-08-16 19:40:43.080,2008-07-31 23:40:59.743,356,2017-11-07 11:12:32.963,2017-11-07 11:12:32.963,Rich B,5292302,1,0,1,1624,<c#><.net><datetime>,How do I calculate someone's age in C#?,417633
4,11,1248,34,<p>Given a specific <code>DateTime</code> valu...,None,3,2009-09-04 13:15:59.820,2008-07-31 23:55:37.967,526,2017-09-04 01:27:10.793,2017-06-04 15:51:19.780,user2370523,6479704,1,0,1,1234,<c#><datetime><time><datediff><relative-time-s...,Calculate relative time in C#,126480


In [6]:
# get size of the table
sql_size = "sp_spaceused 'StackOverflow2017.dbo.Posts'"
data_size = pd.read_sql(sql_size, con)
data_size

,name,rows,reserved,data,index_size,unused
0,Posts,38485046,101325960 KB,100960392 KB,312272 KB,53296 KB


In [7]:
# create two pilot db to test size of columns
data_txt = pd.read_sql("SELECT TOP 10000 Body, Title FROM StackOverflow2017.dbo.Posts", con)
time.sleep(5)
data_num = pd.read_sql("SELECT TOP 10000 Tags, \
    CreationDate, LastActivityDate, ClosedDate, \
    AnswerCount, CommentCount, FavoriteCount, ViewCount, Score, \
    AcceptedAnswerId, OwnerUserId, PostTypeId FROM StackOverflow2017.dbo.Posts", con)

print(sys.getsizeof(data_txt))
print(sys.getsizeof(data_num))

7606023
1247071


Total database size is around 100G. `Post` table contains approximately 85% of `Title` and `Body` information and 15% of all other columns. Propably should process text file separately.

In [8]:
data_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
Tags                2043 non-null object
CreationDate        10000 non-null datetime64[ns]
LastActivityDate    10000 non-null datetime64[ns]
ClosedDate          307 non-null datetime64[ns]
AnswerCount         10000 non-null int64
CommentCount        10000 non-null int64
FavoriteCount       10000 non-null int64
ViewCount           10000 non-null int64
Score               10000 non-null int64
AcceptedAnswerId    10000 non-null int64
OwnerUserId         10000 non-null int64
PostTypeId          10000 non-null int64
dtypes: datetime64[ns](3), int64(8), object(1)
memory usage: 937.6+ KB


In [9]:
data_txt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
Body     10000 non-null object
Title    2043 non-null object
dtypes: object(2)
memory usage: 156.3+ KB


In [10]:
data_posttype = pd.read_sql("SELECT * FROM StackOverflow2017.dbo.PostTypes", con)
data_posttype

,Id,Type
0,1,Question
1,2,Answer
2,3,Wiki
3,4,TagWikiExerpt
4,5,TagWiki
5,6,ModeratorNomination
6,7,WikiPlaceholder
7,8,PrivilegeWiki


In [12]:
data_posttype_freq = pd.read_sql("SELECT T.PostTypeId, COUNT(*) AS freq FROM (SELECT TOP 10000 PostTypeId FROM StackOverflow2017.dbo.Posts) AS T GROUP BY T.PostTypeId", con)
data_posttype_freq

,PostTypeId,freq
0,1,2043
1,2,7957


`Posts` includes **Question**s, **Answer**s and other types of posts. A quick guess from top 10,000 rows of data implies that 20% are questions and 80% are answers (very roughly). It also make sense to pre-process and split by type of posts.

In [13]:
data_question_top10 = pd.read_sql("SELECT TOP 10 * FROM StackOverflow2017.dbo.Posts WHERE PostTypeId = 1", con)
data_question_top10

,Id,AcceptedAnswerId,AnswerCount,Body,ClosedDate,CommentCount,CommunityOwnedDate,CreationDate,FavoriteCount,LastActivityDate,LastEditDate,LastEditorDisplayName,LastEditorUserId,OwnerUserId,ParentId,PostTypeId,Score,Tags,Title,ViewCount
0,4,7,13,<p>I want to use a track-bar to change a form'...,None,1,2012-10-31 16:42:47.213,2008-07-31 21:42:52.667,37,2017-09-27 05:52:59.927,2017-09-27 05:52:59.927,Rich B,3151675,8,0,1,528,<c#><winforms><type-conversion><decimal><opacity>,"While applying opacity to a form, should we us...",33648
1,6,31,5,<p>I have an absolutely positioned <code>div</...,None,0,NaT,2008-07-31 22:08:08.620,9,2016-03-19 06:10:52.170,2016-03-19 06:05:48.487,Rich B,63550,9,0,1,231,<html><css><css3><internet-explorer-7>,Percentage width child element in absolutely p...,15363
2,9,1404,61,<p>Given a <code>DateTime</code> representing ...,None,10,2011-08-16 19:40:43.080,2008-07-31 23:40:59.743,356,2017-11-07 11:12:32.963,2017-11-07 11:12:32.963,Rich B,5292302,1,0,1,1624,<c#><.net><datetime>,How do I calculate someone's age in C#?,417633
3,11,1248,34,<p>Given a specific <code>DateTime</code> valu...,None,3,2009-09-04 13:15:59.820,2008-07-31 23:55:37.967,526,2017-09-04 01:27:10.793,2017-06-04 15:51:19.780,user2370523,6479704,1,0,1,1234,<c#><datetime><time><datediff><relative-time-s...,Calculate relative time in C#,126480
4,13,0,24,<p>Is there any standard way for a Web Server ...,None,4,NaT,2008-08-01 00:42:38.903,127,2017-07-31 11:30:41.573,2016-12-20 19:22:26.510,Rich B,3155639,9,0,1,512,<javascript><html><browser><timezone><timezone...,Determine a User's Timezone,146275
5,14,0,8,"<p>What is the difference between <a href=""htt...",None,1,NaT,2008-08-01 00:59:11.177,51,2017-11-11 03:45:52.287,2017-02-25 17:42:17.810,Rich B,6495084,11,0,1,330,<.net><math>,Difference between Math.Floor() and Math.Trunc...,99255
6,16,12446,5,<p>How do you expose a LINQ query as an ASMX w...,None,0,NaT,2008-08-01 04:59:33.643,13,2016-11-30 08:01:28.050,2016-11-30 08:01:28.050,Chris Fournier,2571493,2,0,1,98,<c#><linq><web-services><.net-3.5>,Filling a DataSet or DataTable from a LINQ que...,76893
7,17,26,10,"<p>How do I store binary data in <a href=""http...",None,0,NaT,2008-08-01 05:09:55.993,16,2016-07-31 15:32:02.653,2011-10-21 21:29:17.160,None,546051,2,0,1,142,<mysql><database><binary-data><data-storage>,Binary Data in MySQL,58963
8,19,531,23,<p>Solutions are welcome in any language. :-) ...,None,14,NaT,2008-08-01 05:21:22.257,74,2017-07-18 22:32:46.757,2015-07-11 12:15:53.303,Chris Jester-Young,635549,13,0,1,257,<performance><algorithm><language-agnostic><un...,What is the fastest way to get the value of π?,39166
9,24,49,6,<p>If I have a <code>trigger</code> <code>befo...,None,0,NaT,2008-08-01 12:12:19.350,19,2016-11-19 21:05:59.207,2016-03-11 08:26:05.063,Jim Anderson,5195227,22,0,1,125,<mysql><database><triggers>,Throw an error in a MySQL trigger,53902


Remove `ParentId` from Questions table. Ignore `CommunityOwnedDate`, `LastEditDate`, `LastEditorDisplayName`, `LastEditorDisplayName`, `LastEditorUserId`. Require `PostTypeId = 1`

In [14]:
data_answer_top10 = pd.read_sql("SELECT TOP 10 * FROM StackOverflow2017.dbo.Posts WHERE PostTypeId = 2", con)
data_answer_top10

,Id,AcceptedAnswerId,AnswerCount,Body,ClosedDate,CommentCount,CommunityOwnedDate,CreationDate,FavoriteCount,LastActivityDate,LastEditDate,LastEditorDisplayName,LastEditorUserId,OwnerUserId,ParentId,PostTypeId,Score,Tags,Title,ViewCount
0,7,0,0,<p>An explicit cast to double isn't necessary....,None,0,NaT,2008-07-31 22:17:57.883,0,2012-10-14 11:50:16.703,2012-10-14 11:50:16.703,None,967315,9,4,2,379,None,None,0
1,12,0,0,"<p>Well, here's how we do it on Stack Overflow...",None,10,2009-09-04 13:15:59.820,2008-07-31 23:56:41.303,0,2014-02-18 14:19:35.770,2014-02-18 14:19:35.770,GateKiller,1420197,1,11,2,312,None,None,0
2,18,0,0,<p>For a table like this:</p>\n\n<pre><code>CR...,None,2,NaT,2008-08-01 05:12:44.193,0,2016-06-02 05:56:26.060,2016-06-02 05:56:26.060,Jeff Atwood,126039,0,17,2,45,None,None,0
3,21,0,0,"<p>Many years ago, to provide an <a href=""http...",None,0,2011-08-16 19:40:43.080,2008-08-01 08:57:27.280,0,2008-08-01 12:35:11.707,2008-08-01 12:35:11.707,Chris Jester-Young,13,13,9,2,31,None,None,0
4,22,0,0,<p>The best way that I know of because of leap...,None,0,2011-08-16 19:40:43.080,2008-08-01 12:07:19.500,0,2008-08-01 15:26:37.087,2008-08-01 15:26:37.087,Nick,17,17,9,2,24,None,None,0
5,26,0,0,<p>The answer by phpguy is correct but I think...,None,0,NaT,2008-08-01 12:16:22.167,0,2016-06-02 05:55:17.060,2016-06-02 05:55:17.060,Geoff Dalgas,126039,48,17,2,109,None,None,0
6,27,0,0,<p>@jeff</p>\n\n<p>IMHO yours seems a little l...,None,0,2009-09-04 13:15:59.820,2008-08-01 12:17:19.357,0,2008-08-01 13:16:49.127,2008-08-01 13:16:49.127,Nick Berardi,17,17,11,2,28,None,None,0
7,29,0,0,<p>There are no HTTP headers that will report ...,None,4,NaT,2008-08-01 12:19:17.417,0,2008-08-01 12:19:17.417,NaT,None,0,19,13,2,68,None,None,0
8,30,0,0,<p>I've had no trouble using the BSD sockets A...,None,0,NaT,2008-08-01 12:22:40.797,0,2008-08-01 12:22:40.797,NaT,None,0,13,25,2,29,None,None,0
9,31,0,0,<p>The parent <code>div</code> needs to have a...,None,0,NaT,2008-08-01 12:22:51.593,0,2016-03-19 06:10:52.170,2016-03-19 06:10:52.170,None,63550,0,6,2,122,None,None,0


Remove `AcceptedAnswerId`, `AnswerCount`, `ClosedDate`, `FavoriteCount`, `ViewCount`, `Tags`, `Title` from Answers table. Ignore `CommunityOwnedDate`, `LastEditDate`, `LastEditorDisplayName`, `LastEditorDisplayName`, `LastEditorUserId`. Require `PostTypeId = 2`

In [11]:
data_user_top10 = pd.read_sql("SELECT TOP 10 * FROM StackOverflow2017.dbo.Users", con)
data_user_top10

,Id,AboutMe,Age,CreationDate,DisplayName,DownVotes,EmailHash,LastAccessDate,Location,Reputation,UpVotes,Views,WebsiteUrl,AccountId
0,-1,"<p>Hi, I'm not really a person.</p>\n\n<p>I'm ...",NaN,2008-07-31 00:00:00.000,Community,890820,None,2008-08-26 00:16:53.810,on the server farm,1,225495,649,http://meta.stackexchange.com/,-1
1,1,"<p><a href=""http://www.codinghorror.com/blog/a...",47.0,2008-07-31 14:22:31.287,Jeff Atwood,1308,None,2017-11-13 11:01:24.840,"El Cerrito, CA",42510,3348,327890,http://www.codinghorror.com/blog/,1
2,2,<p>Developer on the Stack Overflow team. Find...,40.0,2008-07-31 14:22:31.287,Geoff Dalgas,88,None,2017-12-01 22:41:28.983,"Corvallis, OR",3200,642,22508,http://stackoverflow.com,2
3,3,"<p><a href=""http://blog.stackoverflow.com/2009...",38.0,2008-07-31 14:22:31.287,Jarrod Dixon,100,None,2017-12-02 07:08:31.943,"Raleigh, NC, United States",13138,7003,23373,http://jarroddixon.com,3
4,4,<p>I am:</p>\n\n<ul>\n<li>the co-founder and C...,NaN,2008-07-31 14:22:31.317,Joel Spolsky,96,None,2017-12-02 00:28:55.120,"New York, NY",27792,786,71109,http://www.joelonsoftware.com/,4
5,5,"<p>Technical Evangelist at Microsoft, speciali...",47.0,2008-07-31 14:22:31.317,Jon Galloway,34,None,2017-11-30 21:05:02.703,"San Diego, CA",37634,780,10978,http://weblogs.asp.net/jgalloway/,5
6,8,<p>This is a puppet test account I use to vali...,NaN,2008-07-31 21:33:24.057,Eggs McLaren,9,None,2012-10-15 22:00:45.510,,942,12,5914,,6
7,9,<p>Independent software engineer</p>\n,46.0,2008-07-31 21:35:26.517,Kevin Dente,4,None,2017-11-30 23:00:29.633,"Oakland, CA",13314,45,4528,http://weblogs.asp.net/kdente,7
8,10,I'm not takin' my sneakers off!\r\n<br><br>\r\...,38.0,2008-07-31 21:57:06.240,Sneakers O'Toole,0,None,2012-06-19 19:35:43.433,"Lakehead, CA",101,0,3389,http://www.youtube.com/watch?v=qrhNoljPtok,8
9,11,None,NaN,2008-08-01 00:59:11.147,Anonymous User,0,None,2008-08-01 00:59:11.147,None,1666,0,1894,None,561854


In [5]:
data_tags = pd.read_sql("SELECT TOP 10 * FROM StackOverflow2017.dbo.Tags", con)
data_tags

,Id,TagName,Count,ExcerptPostId,WikiPostId
0,1,.net,261481,3624959,3607476
1,2,html,710104,3673183,3673182
2,3,javascript,1519901,3624960,3607052
3,4,css,508419,3644670,3644669
4,5,php,1147808,3624936,3607050
5,8,c,266262,3624961,3607013
6,9,c#,1161054,3624962,3607007
7,10,c++,545591,3624963,3606997
8,12,ruby,189766,3624964,3607043
9,14,lisp,5273,3656743,3656742


In [8]:
data_postlinks = pd.read_sql("SELECT TOP 10 * FROM StackOverflow2017.dbo.PostLinks WHERE LinkTypeId = 3", con)
data_postlinks

,Id,CreationDate,PostId,RelatedPostId,LinkTypeId
0,749670241,2013-02-18 03:03:16.917,70714,355934,3
1,749670246,2013-02-18 03:03:16.917,86947,73713,3
2,749670247,2013-02-18 03:03:16.917,97679,85122,3
3,749670248,2013-02-18 03:03:16.917,99408,1711,3
4,749670249,2013-02-18 03:03:16.917,26925,8127,3
5,749670250,2013-02-18 03:03:16.917,73185,672,3
6,749670251,2013-02-18 03:03:16.917,81150,48935,3
7,749670252,2013-02-18 03:03:16.917,110362,1854,3
8,749670253,2013-02-18 03:03:16.917,117348,506368,3
9,749670254,2013-02-18 03:03:16.917,2366,44965,3


In [7]:
data_LinkType = pd.read_sql("SELECT TOP 10 * FROM StackOverflow2017.dbo.LinkTypes", con)
data_LinkType

,Id,Type
0,1,Linked
1,3,Duplicate


In [18]:
data_comment = pd.read_sql("SELECT TOP 10 * FROM StackOverflow2017.dbo.Comments", con)
data_comment

,Id,CreationDate,PostId,Score,Text,UserId
0,1,2008-09-06 08:07:10.730,35314,41,not sure why this is getting downvoted -- it i...,1
1,2,2008-09-06 08:09:52.330,35314,8,"Yeah, I didn't believe it until I created a co...",3
2,4,2008-09-06 08:42:16.980,35195,0,"I don't see an accepted answer now, I wonder h...",380
3,9,2008-09-06 12:26:30.060,47239,0,"Jonathan: Wow! Thank you for all of that, you...",4550
4,10,2008-09-06 13:38:23.647,45651,6,It will help if you give some details of which...,242
5,12,2008-09-06 13:51:47.843,47428,3,One of the things that make a url user-friendl...,4642
6,14,2008-09-06 14:15:46.897,47481,0,"I agree, both CodeRush and RefactorPro are vis...",4642
7,15,2008-09-06 14:30:40.217,47373,0,Just wanted to mention that this is an excelle...,2495
8,16,2008-09-06 14:42:35.303,47497,1,"Indeed, the only way to do this is get the ser...",4642
9,18,2008-09-06 15:02:40.980,47513,1,"This advice goes against the spirit of CSS, wh...",2515


In [19]:
# create 4 main data query for analysis
sql_question_info = 'SELECT Id, Tags, \
    CreationDate, LastActivityDate, ClosedDate, \
    AnswerCount, CommentCount, FavoriteCount, ViewCount, Score, \
    AcceptedAnswerId, OwnerUserId \
    FROM StackOverflow2017.dbo.Posts WHERE PostTypeId = 1'

sql_answer_info = 'SELECT Id, \
    CreationDate, LastActivityDate, \
    CommentCount, Score, \
    OwnerUserId, ParentId \
    FROM StackOverflow2017.dbo.Posts WHERE PostTypeId = 2'

sql_question_txt = 'SELECT Id, Body, Title \
    FROM StackOverflow2017.dbo.Posts WHERE PostTypeId = 1'

sql_question_txt_2017 = 'SELECT Id, Body, Title \
    FROM StackOverflow2017.dbo.Posts WHERE PostTypeId = 1 AND YEAR(CreationDate) = 2017'

sql_answer_txt = 'SELECT Id, Body\
    FROM StackOverflow2017.dbo.Posts WHERE PostTypeId = 2'

sql_user = 'SELECT Id, AccountId, DisplayName, Age, CreationDate, LastAccessDate, \
    Location, Reputation, UpVotes, DownVotes, Views \
    FROM StackOverflow2017.dbo.Users'

sql_tag = "SELECT * FROM StackOverflow2017.dbo.Tags"

sql_postlink = "SELECT * FROM StackOverflow2017.dbo.PostLinks"

sql_comment = "SELECT * FROM StackOverflow2017.dbo.Comments"

In [10]:
# helper function to write file in chunks
def sql_to_csv(output_file, sql_query, con, chunksize=1000, index_col=None):
    with open(output_file, 'w+', encoding='utf-8') as f:
        df_iter = pd.read_sql(sql_query, con, chunksize=chunksize, index_col="Id")
        for df in df_iter:
            df.to_csv(f, mode='a', header=f.tell()==0)

In [11]:
outpath = "C:/Users/Yuqin/Downloads/StackOverflow_csv/"

In [20]:
%time sql_to_csv(outpath + 'question_info.csv', sql_question_info, con, chunksize=10000, index_col="Id")

Wall time: 9min 12s


In [21]:
time.sleep(10)
%time sql_to_csv(outpath + 'answer_info.csv', sql_answer_info, con, chunksize=10000, index_col="Id")

Wall time: 10min 33s


In [ ]:
time.sleep(10)
%time sql_to_csv(outpath + 'question_txt.csv', sql_question_txt, con, chunksize=10000, index_col="Id")

In [ ]:
time.sleep(10)
%time sql_to_csv(outpath + 'answer_txt.csv', sql_answer_txt, con, chunksize=10000, index_col="Id")

In [15]:
time.sleep(10)
%time sql_to_csv(outpath + 'user.csv', sql_user, con, chunksize=10000, index_col="Id")

Wall time: 3min 15s


In [17]:
%time sql_to_csv(outpath + 'question_txt_2017.csv', sql_question_txt_2017, con, chunksize=10000, index_col="Id")

Wall time: 15min 40s


In [12]:
%time sql_to_csv(outpath + 'tag.csv', sql_tag, con, chunksize=10000, index_col="Id")

Wall time: 786 ms


In [17]:
%time sql_to_csv(outpath + 'postlink.csv', sql_postlink, con, chunksize=10000, index_col="Id")

Wall time: 1min 17s


In [20]:
%time sql_to_csv(outpath + 'comment.csv', sql_comment, con, chunksize=10000, index_col="Id")

Wall time: 31min 6s
